In [1]:
import os
import asyncio
import operator
import requests
from typing import List, Annotated, TypedDict, Union

from pydantic import BaseModel, Field
from IPython.display import display, Markdown
from pprint import pprint
from dotenv import load_dotenv

# LangChain & LangGraph Imports
from langchain_groq import ChatGroq
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.tools.tavily_search import TavilySearchResults

# Load environment variables
load_dotenv()

# Ensure LangSmith Tracing is on if API key is present
if os.getenv("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = "AgenticAI_Workshop" 


In [2]:
# --- 1. CONFIGURATION & INSTRUCTIONS ---
# We preserve your exact instructions as constants for the nodes
PLANNER_INSTRUCTIONS = "You are a helpful research assistant. Given a query, come up with a set of web searches to perform to best answer the query. Output 5 terms to query for."

SEARCH_INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 words. Capture the main points. Write succintly, no need to have complete sentences or good grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

WRITER_INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

PUSH_INSTRUCTIONS = """You are a member of a research team and will be provided with a short summary of a report.
When you receive the report summary, you send a push notification to the user using your tool, informing them that research is complete,
and including the report summary you receive"""

#### SCHEMAS 

In [3]:
class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")

class WebSearchPlan(BaseModel):
    searches: List[WebSearchItem] = Field(description="A list of web searches to perform.")

class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")
    markdown_report: str = Field(description="The final report")
    follow_up_questions: List[str] = Field(description="Suggested topics to research further")

# --- 3. TOOLS ---
@tool
def web_search_tool(query: str) -> str:
    """Search the web for a given term. Use this for research."""
    try:
        # Use Tavily for search (expects TAVILY_API_KEY in env)
        search = TavilySearchResults(max_results=3)
        # Tavily returns a list of dicts, so we convert to string
        return str(search.invoke(query))
    except Exception as e:
        return f"Error performing search: {e}"

@tool
def push_notification_tool(message: str):
    """Send a push notification with this brief message"""
    user_key = os.getenv("PUSHOVER_USER")
    api_token = os.getenv("PUSHOVER_TOKEN")
    if not user_key or not api_token:
        return "Error: PUSHOVER_USER or PUSHOVER_TOKEN not found in environment."
        
    payload = {"user": user_key, "token": api_token, "message": message}
    pushover_url = "https://api.pushover.net/1/messages.json"
    try:
        response = requests.post(pushover_url, data=payload)
        if response.status_code == 200:
             return "success"
        else:
             return f"Failed to send notification: {response.text}"
    except Exception as e:
        return f"Error sending notification: {e}"

#### Graph State & Nodes

In [4]:
#  STATE DEFINITION
class ResearchState(TypedDict):
    # messages tracks the conversation history
    messages: Annotated[List[BaseMessage], add_messages]
    # Specialized fields to hold intermediate data
    query: str
    search_plan: List[WebSearchItem]
    search_results: List[str]
    report: ReportData

# NODE IMPLEMENTATIONS 
model_mini = ChatGroq(model="llama-3.1-8b-instant")
model_large = ChatGroq(model="llama-3.1-8b-instant")

async def planner_node(state: ResearchState):
    """PlannerAgent: Logic to generate the search plan."""
    planner = model_mini.with_structured_output(WebSearchPlan)
    response = await planner.ainvoke([
        SystemMessage(content=PLANNER_INSTRUCTIONS),
        HumanMessage(content=f"Query: {state['query']}")
    ])
    return {
        "search_plan": response.searches,
        "messages": [AIMessage(content=f"Planned {len(response.searches)} searches.")]
    }

async def search_node(state: ResearchState):
    """SearchAgent: Executes processes with autonomous tool loops."""
    search_agent = model_mini.bind_tools([web_search_tool])
    
    async def perform_single_search(item: WebSearchItem):
        # Agent ReAct Loop
        initial_msg = [
            SystemMessage(content=SEARCH_INSTRUCTIONS),
            HumanMessage(content=f"Search term: {item.query}\nReason: {item.reason}")
        ]
        # 1. Ask model
        res1 = await search_agent.ainvoke(initial_msg)
        messages = list(initial_msg) + [res1]
        
        # 2. Check and Execute Tool
        if res1.tool_calls:
            for tc in res1.tool_calls:
                # In this simulated environment, we invoke the tool directly
                if tc['name'] == 'web_search_tool':
                    out = web_search_tool.invoke(tc['args'])
                    messages.append(ToolMessage(content=str(out), tool_call_id=tc['id']))
            
            # 3. Get Summary from Model
            res2 = await search_agent.ainvoke(messages)
            return f"Summary for {item.query}: {res2.content}"
        
        return f"Summary for {item.query}: {res1.content}"

    # Parallel execution
    results = await asyncio.gather(*[perform_single_search(i) for i in state["search_plan"]])
    
    return {
        "search_results": results,
        "messages": [AIMessage(content="Web research completed.")]
    }

async def writer_node(state: ResearchState):
    """WriterAgent: Synthesizes the final report."""
    writer = model_large.with_structured_output(ReportData)
    prompt = f"Original query: {state['query']}\n\nResearch Results:\n" + "\n".join(state["search_results"])
    
    response = await writer.ainvoke([
        SystemMessage(content=WRITER_INSTRUCTIONS),
        HumanMessage(content=prompt)
    ])
    return {
        "report": response,
        "messages": [AIMessage(content="Final report generated.")]
    }

async def push_node(state: ResearchState):
    """PushAgent: Autonomous push notification."""
    pusher = model_mini.bind_tools([push_notification_tool])
    summary = state["report"].short_summary
    
    messages = [
        SystemMessage(content=PUSH_INSTRUCTIONS),
        HumanMessage(content=summary)
    ]
    
    res1 = await pusher.ainvoke(messages)
    messages.append(res1)
    
    if res1.tool_calls:
         for tc in res1.tool_calls:
             if tc['name'] == 'push_notification_tool':
                 out = push_notification_tool.invoke(tc['args'])
                 messages.append(ToolMessage(content=str(out), tool_call_id=tc['id']))
         
         res2 = await pusher.ainvoke(messages)
         return {"messages": [AIMessage(content="Notification pushed and confirmed.")]}
    
    return {"messages": [AIMessage(content="Notification step completed (no call).")]}


####  Assembly & Execution

In [5]:
# GRAPH ASSEMBLY 
builder = StateGraph(ResearchState)

builder.add_node("planner", planner_node)
builder.add_node("researcher", search_node)
builder.add_node("writer", writer_node)
builder.add_node("notifier", push_node)

builder.add_edge(START, "planner")
builder.add_edge("planner", "researcher")
builder.add_edge("researcher", "writer")
builder.add_edge("writer", "notifier")
builder.add_edge("notifier", END)

# Compile with a checkpointer for LangSmith thread-level tracing
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [6]:
# 7. RUNTIME EXECUTION
async def run_workflow(user_query: str):
    print(f"--- Starting Research: {user_query} ---")
    
    inputs = {"query": user_query, "messages": [HumanMessage(content=user_query)]}
    config = {"configurable": {"thread_id": "workshop_user_1"}}
    
    # We stream the values to show progress in the workshop
    async for event in graph.astream(inputs, config=config, stream_mode="values"):
        if "messages" in event:
            last_msg = event["messages"][-1]
            print(f"[{last_msg.type.upper()}]: {last_msg.content[:100]}...")
            if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                print(f"  [TOOL CALL]: {last_msg.tool_calls[0]['name']}")

    # Final Output Rendering
    final_state = await graph.aget_state(config)
    report = final_state.values["report"]
    
    display(Markdown("# Final Research Report"))
    display(Markdown(report.markdown_report))
    print("\nFollow-up Questions:")
    for q in report.follow_up_questions:
        print(f"- {q}")

# Launch
await run_workflow("What are the most popular and successful AI Agent frameworks in May 2025")

--- Starting Research: What are the most popular and successful AI Agent frameworks in May 2025 ---
[HUMAN]: What are the most popular and successful AI Agent frameworks in May 2025...
[AI]: Planned 5 searches....


C:\Users\ADEGOKE\AppData\Local\Temp\ipykernel_17516\2152884298.py:19: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search = TavilySearchResults(max_results=3)


[AI]: Web research completed....


BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=ReportData> {"short_summary": "This report outlines the most popular and successful AI Agent frameworks as of May 2025, including their key features, strengths, and market share. The report also discusses the current market trends and future projections for the AI Agent framework market. The top agentic AI frameworks in 2025 are LangChain, LangGraph, AutoGen, AgentGPT, and AutoGPT, with LangChain and LangGraph being the most widely used. The market is expected to grow rapidly, with a projected market size of $7.38 billion by the end of 2025 and an estimated $52.62 billion by 2030.", "markdown_report": "# AI Agent Frameworks Report\\n\\n## Table of Contents\\n\\n1. Introduction\\n2. Popular AI Agent Frameworks\\n3. Successful AI Agent Framewor